In [4]:
# !pip install --upgrade pip
# !pip install OpenSSL.SSL
!pip install catboost 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 11.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 18.3 MB/s eta 0:00:00m eta 0:00:010:01:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Распознование датасета фотографий отжиманий, (нахождение ключевых точек на фотографиях) 

In [3]:
from ultralytics import YOLO
import pandas as pd
import cv2

# Загрузка модели
model = YOLO('yolov8x-pose-p6.pt')

# Загрузка изображения
# image_path = 'data/train/youtube-101_jpg.rf.8661eb05f9be6162a65c358cea634199.jpg'
# image = cv2.imread(image_path)
# detect_image(image_path)
def detect_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки изображения: {image_path}")
        return None
    # Распознавание ключевых точек
    results = model(image,save=False)
    if not results:
        print("Объекты не обнаружены.")
        return None
    result= results[0]
    x1 = result.boxes.xyxy[0][0].item()
    y1 = result.boxes.xyxy[0][1].item()
    x2 = result.boxes.xyxy[0][2].item()
    y2 = result.boxes.xyxy[0][3].item()
    w = x2 - x1  # ширина bounding box
    h = y2 - y1  # высота bounding box
    ans = {}
    for i, (x, y) in enumerate(result.keypoints.xy[0]):
        if x.item()!=0:
            ans[f"{i}_x"] = (x.item()-x1)/w
            ans[f"{i}_y"] = (y.item()-y1)/h
        else:
            ans[f"{i}_x"] = x.item()
            ans[f"{i}_y"] = y.item()
    return ans

base_path = 'data/train/'
image_data = pd.read_csv('data/train/_classes.csv')
results_df = pd.DataFrame()
for index, row in image_data.iterrows():
    image_path = base_path + row['filename'].strip()
    keypoints = detect_image(image_path)
    keypoints['push_down'] = row[' push_down']  # Обратите внимание на пробел в названии столбца
    keypoints['push_up'] = row[' push_up']
    if keypoints:
        results_df = results_df.append(pd.DataFrame(keypoints, index=[0]), ignore_index=True)

base_path = 'data/test/'
image_data = pd.read_csv('data/test/_classes.csv')
for index, row in image_data.iterrows():
    image_path = base_path + row['filename'].strip()
    keypoints = detect_image(image_path)
    keypoints['push_down'] = row[' push_down']  # Обратите внимание на пробел в названии столбца
    keypoints['push_up'] = row[' push_up']
    if keypoints:
        results_df = results_df.append(pd.DataFrame(keypoints, index=[0]), ignore_index=True)
        
        
base_path = 'data/valid/'
image_data = pd.read_csv('data/valid/_classes.csv')
for index, row in image_data.iterrows():
    image_path = base_path + row['filename'].strip()
    keypoints = detect_image(image_path)
    keypoints['push_down'] = row[' push_down']  # Обратите внимание на пробел в названии столбца
    keypoints['push_up'] = row[' push_up']
    if keypoints:
        results_df = results_df.append(pd.DataFrame(keypoints, index=[0]), ignore_index=True)
        


100%|████████████████████████████████████████| 190M/190M [00:15<00:00, 12.9MB/s]

0: 768x1280 1 person, 1418.7ms
Speed: 6.3ms preprocess, 1418.7ms inference, 1.3ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1415.7ms
Speed: 7.7ms preprocess, 1415.7ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1416.3ms
Speed: 9.3ms preprocess, 1416.3ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1416.2ms
Speed: 6.8ms preprocess, 1416.2ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1417.2ms
Speed: 7.4ms preprocess, 1417.2ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1416.8ms
Speed: 8.0ms preprocess, 1416.8ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1413.4ms
Speed: 9.3ms preprocess, 1413.4ms inference, 1.4ms postprocess per image at shape (1, 3, 768, 1


0: 768x1280 1 person, 1442.8ms
Speed: 7.4ms preprocess, 1442.8ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1442.7ms
Speed: 8.6ms preprocess, 1442.7ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1439.4ms
Speed: 8.8ms preprocess, 1439.4ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1440.9ms
Speed: 6.9ms preprocess, 1440.9ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1440.8ms
Speed: 8.3ms preprocess, 1440.8ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1441.2ms
Speed: 8.6ms preprocess, 1441.2ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1442.9ms
Speed: 8.5ms preprocess, 1442.9ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1442.2ms
Speed: 7.9ms preprocess, 1442.2ms inference,

Speed: 7.7ms preprocess, 1453.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1455.3ms
Speed: 8.1ms preprocess, 1455.3ms inference, 1.6ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1453.9ms
Speed: 7.7ms preprocess, 1453.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1452.8ms
Speed: 7.3ms preprocess, 1452.8ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1456.8ms
Speed: 7.3ms preprocess, 1456.8ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1454.6ms
Speed: 8.3ms preprocess, 1454.6ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1454.9ms
Speed: 7.6ms preprocess, 1454.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1454.6ms
Speed: 9.3ms preprocess, 1454.6ms inference, 1.8ms postprocess per image at 


0: 768x1280 1 person, 1485.2ms
Speed: 7.5ms preprocess, 1485.2ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1482.1ms
Speed: 7.3ms preprocess, 1482.1ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1483.2ms
Speed: 8.2ms preprocess, 1483.2ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1482.9ms
Speed: 7.5ms preprocess, 1482.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1482.2ms
Speed: 7.3ms preprocess, 1482.2ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1480.9ms
Speed: 7.4ms preprocess, 1480.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1483.6ms
Speed: 7.3ms preprocess, 1483.6ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1480.7ms
Speed: 7.3ms preprocess, 1480.7ms inference,

Speed: 7.9ms preprocess, 1512.2ms inference, 2.9ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1508.9ms
Speed: 8.7ms preprocess, 1508.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.0ms
Speed: 8.6ms preprocess, 1511.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.3ms
Speed: 9.1ms preprocess, 1511.3ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1508.0ms
Speed: 8.8ms preprocess, 1508.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.0ms
Speed: 8.7ms preprocess, 1510.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.3ms
Speed: 8.4ms preprocess, 1510.3ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.6ms
Speed: 8.8ms preprocess, 1509.6ms inference, 1.7ms postprocess per image at 


0: 768x1280 1 person, 1512.0ms
Speed: 7.6ms preprocess, 1512.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.5ms
Speed: 8.8ms preprocess, 1510.5ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.1ms
Speed: 8.7ms preprocess, 1511.1ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.3ms
Speed: 7.4ms preprocess, 1509.3ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.9ms
Speed: 8.6ms preprocess, 1511.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.5ms
Speed: 8.9ms preprocess, 1510.5ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.0ms
Speed: 8.5ms preprocess, 1510.0ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.9ms
Speed: 8.1ms preprocess, 1510.9ms inference,

Speed: 7.4ms preprocess, 1511.7ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.1ms
Speed: 7.4ms preprocess, 1509.1ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.1ms
Speed: 8.8ms preprocess, 1510.1ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1508.3ms
Speed: 8.5ms preprocess, 1508.3ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1512.5ms
Speed: 7.4ms preprocess, 1512.5ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.9ms
Speed: 7.9ms preprocess, 1509.9ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.5ms
Speed: 7.4ms preprocess, 1510.5ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1508.6ms
Speed: 7.9ms preprocess, 1508.6ms inference, 1.8ms postprocess per image at 


0: 768x1280 1 person, 1510.1ms
Speed: 7.5ms preprocess, 1510.1ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.8ms
Speed: 8.0ms preprocess, 1509.8ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.2ms
Speed: 8.6ms preprocess, 1511.2ms inference, 1.9ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.8ms
Speed: 8.6ms preprocess, 1510.8ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1511.0ms
Speed: 8.0ms preprocess, 1511.0ms inference, 3.0ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.1ms
Speed: 11.9ms preprocess, 1509.1ms inference, 1.9ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1509.7ms
Speed: 8.0ms preprocess, 1509.7ms inference, 1.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 1 person, 1510.2ms
Speed: 7.5ms preprocess, 1510.2ms inference

In [5]:
# image = 'data/train/youtube-101_jpg.rf.8661eb05f9be6162a65c358cea634199.jpg'
# results = model(image,save=False)
# results[0].boxes.xyxy[0][0].item()
# import numpy as np
# key_points_frame = pd.DataFrame()

# data = {}

# for i, (x, y) in enumerate(result.keypoints.xy[0]):
#     data[f"{i}_x"] = [x.item()]
#     data[f"{i}_y"] = [y.item()]
results_df
results_df.to_csv('data/keypoints_results_x_large.csv', index=False)
# key_points_frame = pd.DataFrame(data)
# key_points_frame


## Обучение класификатора

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from joblib import dump

# Загрузка данных
data = pd.read_csv('data/keypoints_results_x.csv')

# Предполагаем, что первые N-2 столбцы - это признаки, а последние два - это метки
X = data.iloc[:, :-2]
y = data[['push_up', 'push_down']]  # Или data['push_down'] в зависимости от того, что вы предсказываете

# Разделение данных на тренировочные и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Создание и обучение модели
model_random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
model_random_forest.fit(X_train, y_train)
model_cat_boost = CatBoostClassifier(iterations=50,
                           depth=10,
                           learning_rate=1,
                           loss_function='MultiLogloss',
                           verbose=True)

model_cat_boost.fit(X_train, y_train)
# Оценка модели
predictions_random_forest = model_random_forest.predict(X_test)
accuracy_push_up = accuracy_score(y_test['push_up'], predictions_random_forest[:, 0])
accuracy_push_down = accuracy_score(y_test['push_down'], predictions_random_forest[:, 1])
print(f"Accuracy for push_up random_forest: {accuracy_push_up}")
print(f"Accuracy for push_down random_forest: {accuracy_push_down}")
dump(model_random_forest, 'push_up_checker_random_forest.joblib')

predictions_cat_boost = model_cat_boost.predict(X_test)
accuracy_push_up = accuracy_score(y_test['push_up'], predictions_cat_boost[:, 0])
accuracy_push_down = accuracy_score(y_test['push_down'], predictions_cat_boost[:, 1])
print(f"Accuracy for push_up random_forest: {accuracy_push_up}")
print(f"Accuracy for push_down random_forest: {accuracy_push_down}")
dump(model_cat_boost, 'push_up_checker_cat_boost.joblib')



0:	learn: 0.1128487	total: 71.1ms	remaining: 3.48s
1:	learn: 0.0505159	total: 143ms	remaining: 3.43s
2:	learn: 0.0302583	total: 211ms	remaining: 3.31s
3:	learn: 0.0229698	total: 281ms	remaining: 3.24s
4:	learn: 0.0177309	total: 358ms	remaining: 3.22s
5:	learn: 0.0133315	total: 426ms	remaining: 3.12s
6:	learn: 0.0106353	total: 500ms	remaining: 3.07s
7:	learn: 0.0095914	total: 575ms	remaining: 3.02s
8:	learn: 0.0082013	total: 646ms	remaining: 2.94s
9:	learn: 0.0070370	total: 740ms	remaining: 2.96s
10:	learn: 0.0067961	total: 837ms	remaining: 2.97s
11:	learn: 0.0056205	total: 936ms	remaining: 2.96s
12:	learn: 0.0051976	total: 1.03s	remaining: 2.93s
13:	learn: 0.0045347	total: 1.11s	remaining: 2.86s
14:	learn: 0.0041511	total: 1.18s	remaining: 2.76s
15:	learn: 0.0037583	total: 1.26s	remaining: 2.68s
16:	learn: 0.0034700	total: 1.33s	remaining: 2.59s
17:	learn: 0.0031891	total: 1.4s	remaining: 2.5s
18:	learn: 0.0029358	total: 1.48s	remaining: 2.42s
19:	learn: 0.0026910	total: 1.56s	remainin

['push_up_checker_cat_boost.joblib']

In [1]:
import cv2
cv2.imshow('Test Window', cv2.imread('your_image.jpg'))
cv2.waitKey(0)
cv2.destroyAllWindows()


[ WARN:0@0.076] global loadsave.cpp:248 findDecoder imread_('your_image.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
